# Assignment <span style="color:red">option Four</span> - News Categorization  using PyTorch 
Download the dataset from https://www.kaggle.com/uciml/news-aggregator-dataset and develop a news classification or categorization model. The dataset contain only titles of a news item and some metadata. The categories of the news items include one of: –<span  style="color:red"> b</span> : business – <span  style="color:red">t</span> : science and technology – <span  style="color:red">e</span> : entertainment and –<span  style="color:red">m</span> : health. 

1. Prepare training and test dataset: Split the data into training and test set (80% train and 20% test). Make sure they are balanced, otherwise if all <span  style="color:red">b</span> files are on training, your model fails to predict <span  style="color:red">t</span> files in test.
2. Binary classification: produce training data for each two categories, such as <span  style="color:red">b </span> and <span  style="color:red"> t</span>, <span  style="color:red">b</span> and <span  style="color:red"> m</span>, <span  style="color:red">e</span> and <span  style="color:red">t</span> and so on. Evaluate the performance and report which categories are easier for the models.
3. Adapt the Text Categorization PyTorch code (see above) and evaluate the performance of the system for these task
4. Use a pre-trained embeddings and compare your result. When you use pre-trained embeddings, you have to average the word embeddings of each tokens in ach document to get the unique representation of the document. DOC_EMBEDDING = (TOKEN1_EMBEDDING + ... + TOKENn_EMBEDDING). You can also use some of the <span  style="color:red">spacy/FLAIR </span>document embedding methods
6. Report the recall, precision, and F1 scores for both binary and multi-class classification.
 

1. Prepare training and test dataset: Split the data into training and test set (80% train and 20% test). Make sure they are balanced, otherwise if all <span  style="color:red">b</span> files are on training, your model fails to predict <span  style="color:red">t</span> files in test.

In [15]:
# %pip install torch torchvision


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Daten einlesen
# data = pd.read_csv("./data/uci-news-aggregator.csv")
data = pd.read_csv("./data/uci-news-aggregator.csv")
# Daten in Features (X) und Labels (y) aufteilen: Wenn mehr features, dann die weiteren hinzufügen
X = data[['TITLE', 'PUBLISHER', 'CATEGORY']]
X = data[['TITLE', 'CATEGORY']]
y = data['CATEGORY']

TEST_SIZE = 0.2

# Aufteilung in Trainings- und Testdaten. Der Parameter stratify führt dazu, dass das feature "Category" gleich aufgeteilt sind
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42, stratify=data['CATEGORY'])

print("Trainingsdaten: ", len(X_train))
print("Testdaten: ", len(X_test))

Trainingsdaten:  337935
Testdaten:  84484


2. Binary classification: produce training data for each two categories, such as <span  style="color:red">b </span> and <span  style="color:red"> t</span>, <span  style="color:red">b</span> and <span  style="color:red"> m</span>, <span  style="color:red">e</span> and <span  style="color:red">t</span> and so on. Evaluate the performance and report which categories are easier for the models.

–<span  style="color:red"> b</span> : business – <span  style="color:red">t</span> : science and technology – <span  style="color:red">e</span> : entertainment and –<span  style="color:red">m</span> : health

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Laden Sie den Datensatz herunter und extrahieren Sie ihn in den Root-Ordner.

# Annahme: Der Datensatz befindet sich in der Datei 'newsCorpora.csv' im Root-Ordner.
# Stellen Sie sicher, dass Sie die Datei im richtigen Format erhalten (CSV).

# Laden Sie den Datensatz
dataset_path = '/content/gdrive/MyDrive/uci-news-aggregator.csv'
df = pd.read_csv(dataset_path)
df = df[["TITLE", "CATEGORY"]]

# 1. Vorbereitung des Trainings- und Testdatensatzes
# Aufteilung in Trainings- und Testset (80% Train und 20% Test), sicherstellen, dass sie ausgeglichen sind
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['CATEGORY'], random_state=42)

# 2. Binäre Klassifikation für jede Kombination von zwei Kategorien
categories = ['b', 't', 'e', 'm']

for i in range(len(categories)):
    for j in range(i + 1, len(categories)):
        category_pair = [categories[i], categories[j]]
        print(f"\nBinary Classification: {category_pair}")

        # Filtern Sie die Daten für die ausgewählten Kategorien
        train_data = train_df[train_df['CATEGORY'].isin(category_pair)]
        test_data = test_df[test_df['CATEGORY'].isin(category_pair)]

        # Extrahieren Sie Titel und Labels
        X_train, y_train = train_data['TITLE'], train_data['CATEGORY']
        X_test, y_test = test_data['TITLE'], test_data['CATEGORY']

        # Textvektorisierung
        vectorizer = CountVectorizer()
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)

        # Modelltraining (hier: Naive Bayes)
        model = MultinomialNB()
        model.fit(X_train_vec, y_train)

        # Vorhersagen
        y_pred = model.predict(X_test_vec)

        # Evaluierung der Leistung
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        print(f"Accuracy: {accuracy:.2f}")
        print("Classification Report:")
        print(report)
        print("---------------------")




: 

: 

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Read the dataset
# data = pd.read_csv("/content/gdrive/MyDrive/uci-news-aggregator.csv")
data = pd.read_csv("./data/uci-news-aggregator.csv")

# Define categories
categories = ['b', 't', 'e', 'm']

# Create combinations of categories for binary classification
category_combinations = [(categories[i], categories[j]) for i in range(len(categories)) for j in range(i+1, len(categories))]

# Iterate through each category combination
for category_pair in category_combinations:
    category1, category2 = category_pair

    # Create a binary dataset for the current category pair
    binary_data = data[(data['CATEGORY'] == category1) | (data['CATEGORY'] == category2)]
    print(binary_data)

    # Split the binary dataset into training and test sets
    X = binary_data['TITLE']
    y = binary_data['CATEGORY']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Vectorize the titles using TF-IDF
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Train a classifier (using Multinomial Naive Bayes as an example)
    classifier = MultinomialNB()
    classifier.fit(X_train_tfidf, y_train)

    # Make predictions on the test set
    predictions = classifier.predict(X_test_tfidf)

    # Evaluate performance
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)

    # Report results
    print(f"Category Pair: {category1} vs {category2}")
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", report)
    print("-------------------------------------------")
    exit()


            ID                                              TITLE  \
0            1  Fed official says weak data caused by weather,...   
1            2  Fed's Charles Plosser sees high bar for change...   
2            3  US open: Stocks fall after Fed official hints ...   
3            4  Fed risks falling 'behind the curve', Charles ...   
4            5  Fed's Plosser: Nasty Weather Has Curbed Job Gr...   
...        ...                                                ...   
419631  420150  iPad Air 2 And iPad Mini 3 Release Date, Specs...   
419632  420151     Apple rumored to launch its new iPad this fall   
419633  420152  For many, the once-hyped tablet has become dea...   
419634  420153  iPad Air 2 and iPad Mini 3 Release Date, Price...   
419635  420154  Can the iPad Air 2 Bring Apple, Inc.'s Tablet ...   

                                                      URL  \
0       http://www.latimes.com/business/money/la-fi-mo...   
1       http://www.livemint.com/Politics/H2EvwJSK

: 